# Probability-Probability plots

This notebook includes code to reproduce Figure 4 and Figures B1-3 which all show P-P plots for the different samplers and combinations of detectors.

We use a function from `gw_smc_utils` to produce the P-P plots from the saved credible levels rather than recomputing them.

Import the various modules we need

In [ ]:
from pathlib import Path
from gw_smc_utils.plotting import set_style, pp_plot_from_credible_levels
import pandas as pd
import matplotlib.pyplot as plt
import re

set_style()

Path("figures").mkdir(exist_ok=True)

Specify the path to the data release and directory with the pre-computed credible levels.

In [ ]:

data_release_path = Path("../data_release/gw_smc_data_release_core/")
credible_levels_path = data_release_path / "simulated_data" / "pp_tests" / "credible_levels"


We specify which parameters to include in the P-P plot.

In [ ]:
keys = [
    "chirp_mass",
    "mass_ratio",
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "luminosity_distance",
    "dec",
    "ra",
    "theta_jn",
    "psi",
    "geocent_time",
    "phase",
]

We then produce all four P-P plots for `dynesty` and `pocomc`.

We use `re` to determine the sampler and number of detectors from the name of the file.

In [ ]:
credible_level_files = sorted(credible_levels_path.glob("*.hdf5"))
for clf in credible_level_files:
    sampler = re.search(r"(dynesty|pocomc)", clf.name).group(0)
    ndet = re.search(r"(\d+)det", clf.name).group(0)
    print(f"\nProducing P-P plot for {sampler} in {ndet}")
    if not sampler or not ndet:
        raise ValueError(f"Could not extract sampler or ndet from {clf.name}")
    credible_levels = pd.read_hdf(clf, "credible_levels")
    fig, pvalues = pp_plot_from_credible_levels(
        credible_levels,
    )
    fig.savefig(f"figures/pp_test_{sampler}_{ndet}.pdf")
    plt.show()